In [46]:
import re
import unicodedata
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import pyodbc
import nltk
from nltk.tag import pos_tag
from googletrans import Translator
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import sparknlp
from rake_nltk import Rake

In [47]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dryne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dryne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\dryne\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dryne\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [48]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-KLLTOUUR\SQLEXPRESS;'
                      'Database=Dashboard;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
cursor2=conn.cursor()

In [49]:
query2 = "SELECT * FROM EY_jobs"

cursor2.execute(query2)
dataj = cursor2.fetchall()

columns = [column[0] for column in cursor2.description]

dfj = pd.DataFrame(columns=columns)

for rowj in dataj:
    dfj = pd.concat(
        [dfj, pd.DataFrame([dict(zip(columns, rowj))])], ignore_index=True)

In [50]:
translator = Translator(service_urls=['translate.google.com'])


# Function to translate text to English
def translate_text(text):
    translation = translator.translate(text, dest='en')
    return translation.text

# Apply translation to the DataFrame column
dfj["Description"] = dfj["Description"].apply(translate_text)
dfj["Title"] = dfj["Title"].apply(translate_text)


In [51]:
jobs=["the","opportunity","ey","job","description"]
clean_keywords=["apply","integrate","using","existing","without","starts","french","end","products"]
sw=[",",".","'","!","?","@","#","[]","·","people","clients","1","2","3","4","5","6","7","8","9",
           "ey","freedom","work","closely","150","review","equivalent","excellence","employer","english","skills","sized","would","us","degree","long","oral","regulatory","team","mastery","year",
            "consultants","use","world","areas","excellent","offer","needs","demonstrate","face","perfect","high","graduate","covers"
            ,"etc","tunisian","(",")","•","countries",").","speed","especially","multiple","mentioned","study","policy","pressure","cca","french","employees",".).","give","good","future","every","level","two","stay","first","enjoy","life","africa","within","written","thus","also","years","deliver","including","monthly","hands","today","good","top","key","esc","’","new"]
stop_words = set(stopwords.words('english'))
stop_words.update(sw)

In [52]:
def contains_stopword(string, stopwords):
    words=string.split()
    for word in stopwords:
        if word in words  :
            return True
    return False

def clean_jobs(text):
    words=text.split()
    b=True
    i=0
    while(i < len(words) and b==True):
        
        if words[i].lower() in jobs : 
            words.remove(words[i])
        else:
            b=False
            i+=1
        
    text = " ".join(words)
    text=text.capitalize()
    return text 


def clean_keyword(text):
    words=text.split()
    b=True
    i=0
    while(i < len(words) and b==True):
        
        if words[i].lower() in clean_keywords : 
            words.remove(words[i])
        else:
            b=False
            i+=1
        
    text = " ".join(words)
    text=text.capitalize()
    text="#"+text
    text = text.replace(" ", "_")
    return text 


def extract_keywords(texte,i): 
    l=[]
    keywords=[]
    r=Rake()
    r.extract_keywords_from_text(texte)
    
    for score,keyword in r.get_ranked_phrases_with_scores():
       
        if (((score>=3.2) and (score<=5.5)) or (score>=17.0 and score<=20)) and (contains_stopword(keyword,stop_words)==False) and (keyword not in keywords):
            
            l.append((i,clean_keyword(keyword),score))
            keywords.append(keyword)
        
    return l



In [53]:
for i in range(len(dfj["Description"])):    
    dfj["Description"][i]=clean_jobs(dfj["Description"][i])

In [56]:
data=[]
for i in range(len(dfj["Description"])):
    data.append(extract_keywords(dfj["Description"][i],dfj["Id"][i]))
my_list=[item for sublist in data for item in sublist]
data_frame=pd.DataFrame(my_list,columns=['Id','Keyword','Score'])
print("Cleaned text : \n")
print(dfj["Description"][0])
print("\nKeywords using my algorithm : \n")
keywords_id_15 = data_frame.loc[data_frame['Id'] == 15, 'Keyword']
print(keywords_id_15)

Cleaned text : 

As part of a team led by an experienced manager, you will be required to carry out tasks such as : - supervision of accounting and payroll files - management of clients' monthly payroll - preparing tax returns - reporting to customers - preparation of financial statements and tax returns - participate in setting up internal work processes - customer relationship management (email exchanges, video-conferences and face-to-face meetings). your profile -degree: bachelor's degree in accounting (ihec, isce, etc.) or master's degree in cca related to accounting - minimum 3 years' experience in an accounting firm or company - management of tunisian payroll and social security declarations -management of monthly, quarterly and annual tax returns -preparation of taxable income table - preparation of periodic reports and tax returns - basic knowledge of ifrs -proficiency in sage accounting and payroll - proficiency in pack office, particularly excel - perfect command of french an